In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

from functools import partial

import tensorflow as tf
from sklearn.metrics import confusion_matrix

from iris_data import train, x_train, y_train, \
                      test, x_test, y_test, \
                      SPECIES, LARGE

In [3]:
tf.set_random_seed(1)
tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
# construct inputs
def input_fn(df, batch_size=1, shuffle=True, num_epochs=None):
    features = df.to_dict(orient='list')
    features = {k: np.array(v) for k, v in features.items()}
    try:
        labels = features['Species']
        del features['Species']
        ds = tf.data.Dataset.from_tensor_slices((features, labels))
    except KeyError:
        ds = tf.data.Dataset.from_tensor_slices(features)

    if shuffle:
        ds = ds.shuffle(buffer_size=LARGE)
    
    ds = ds.repeat(num_epochs).batch(batch_size)
    return ds.make_one_shot_iterator().get_next()

predict_input_fn = partial(input_fn, num_epochs=1, shuffle=False)

In [5]:
# create model
col = [
    tf.feature_column.numeric_column('SepalLength'), 
    tf.feature_column.numeric_column('SepalWidth'), 
    tf.feature_column.numeric_column('PetalLength'), 
    tf.feature_column.numeric_column('PetalWidth')
]

optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)

model = tf.estimator.DNNClassifier(
    feature_columns=col,
    hidden_units=[2],
    optimizer=optimizer,
    n_classes=len(SPECIES)
)

In [6]:
# train
model.train(input_fn=lambda: input_fn(train, batch_size=1, num_epochs=20))

In [7]:
# confusion matrix
y_test_pred = model.predict(input_fn=lambda: predict_input_fn(test))
y_test_pred = np.array([item['class_ids'][0] for item in y_test_pred])
confusion_matrix(y_test, y_test_pred)

array([[ 8,  0,  0],
       [ 0, 13,  1],
       [ 0,  0,  8]])